In [1]:
# ライブラリのインポート
import numpy as np
import pandas as pd
import os

# データフレーム読み込み
train_df = pd.read_csv("/kaggle/input/playground-series-s5e8/train.csv")
test_df = pd.read_csv("/kaggle/input/playground-series-s5e8/test.csv")

# データ結合
all_df = pd.concat([train_df,test_df],axis=0,ignore_index=True)
all_df_NN = all_df.copy()
max_row = len(all_df)

In [2]:
# all_df.info() # 特徴量、欠損、型確認

In [3]:
# test_df.head()

In [4]:
# all_df.head(1)

### 数値列のカテゴリ化を行う

### 新規特徴量を作成する

In [5]:
# (1) 住宅ローン + ローン
all_df["housing_loan"] = all_df["housing"].astype(str) + "_" + all_df["loan"].astype(str)

# (2) コンタクト時間 x 年齢
all_df["duration_x_age"] = all_df["duration"] * all_df["age"]

# (3) sin,cos(コンタクト時間)
all_df['duration_sin'] = np.sin(2*np.pi * all_df['duration'] / 400)
all_df['duration_cos'] = np.cos(2*np.pi * all_df['duration'] / 400)

# (4) monthを数値に直し周期的に使う
month_map = {'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4,
    'may': 5, 'jun': 6, 'jul': 7, 'aug': 8,
    'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12}
all_df['month_num'] = all_df['month'].map(month_map).astype('int')
all_df['month_sin'] = np.sin(2 * np.pi * all_df['month_num'] / 12)
all_df['month_cos'] = np.cos(2 * np.pi * all_df['month_num'] / 12)

# (5) コンタクト時間をカテゴリ化
all_df['duration_bin'] = pd.cut(
    all_df['duration'],
    bins=[0, 60, 300, 600, 900, float('inf')],
    labels=['short', 'medium', 'long', 'very_long', 'extreme'],
    right=False)
all_df['duration_bin'] = all_df['duration_bin'].astype("object")

# (6) 連絡手段 + 年齢
all_df['age_group'] = pd.cut(
    all_df['age'],
    bins=[0, 30, 45, 60, 100],
    labels=['young', 'mid', 'senior', 'elder'])
all_df["contact_age"] = all_df["contact"].astype(str) + "_" + all_df["age_group"].astype(str)
all_df = all_df.drop("age_group",axis=1)

# (7) sin,cos(pdays)
all_df['pdays_sin'] = np.sin(2*np.pi * all_df['pdays'] / 90)
all_df['pdays_cos'] = np.cos(2*np.pi * all_df['pdays'] / 90)

In [6]:
# dayが5-10日はy=0が多い
# all_df["day_cls"] = all_df["day"].copy()
# all_df.loc[all_df["day"]%10<5,"day_cls"] = "1stHalf"
# all_df.loc[all_df["day"]%10>=5,"day_cls"] = "2ndHalf"
# all_df["day_cls"] = all_df["day_cls"].astype(str)

# sin,cos(10days)
# all_df['days_sin/5'] = np.sin(2*np.pi * all_df['day'] / 5)
# all_df['days_cos/5'] = np.cos(2*np.pi * all_df['day'] / 5)

# sin,cos(15days)
# all_df['days_sin/15'] = np.sin(2*np.pi * all_df['pdays'] / 15)
# all_df['days_cos/15'] = np.cos(2*np.pi * all_df['pdays'] / 15)

### カテゴリ化した元は数値の列を削除する

In [7]:
# # カテゴリ列削除
# RMV = ["age_cls","balance_cls","duration_cls","campaign_cls","pdays_cls","previous_cls"]
# all_df = all_df.drop(RMV,axis=1)

In [8]:
# 数値列とカテゴリ列を取得
num_col = []
cat_col = []

# train_df = train_df.drop(["id","y"],axis=1)

all_df2 = all_df.drop(["id","y"],axis=1)

for col in all_df2.columns:
    if all_df[col].dtypes!="object":
        num_col.append(col)
    else:
        cat_col.append(col)
        # all_df[col] = all_df[col].astype("category")

In [9]:
# 訓練データとテストデータに分離
train = all_df[:len(train_df)]
test = all_df[len(train_df):]

# 訓練データをx,yに分割
X_train = train.drop(["id","y"],axis=1)
y_train = train["y"]
X_test = test.drop(["id","y"],axis=1)

In [10]:
# # 相関係数を確認する
# import seaborn as sns
# import matplotlib.pyplot as plt

# train_corr = X_train.drop(cat_col,axis=1).corr()
# sns.clustermap(train_corr,annot=True,fmt=".2f",cmap="bwr")
# plt.show()

In [11]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# 辞書にLabelEncoderを保持
label_encoders = {}

# カテゴリ列だけエンコード
X_train_cat = pd.DataFrame()
X_test_cat = pd.DataFrame()

for col in cat_col:
    le = LabelEncoder()
    le.fit(X_train[col])

    X_train_cat[col] = le.transform(X_train[col])
    X_test_cat[col] = le.transform(X_test[col])

    label_encoders[col] = le

# 数値列はそのままコピー
X_train_num = X_train[num_col].copy()
X_train_num = X_train_num.reset_index()
X_test_num = X_test[num_col].copy()
X_test_num = X_test_num.reset_index()

# 最後に結合（カテゴリ + 数値）
X_train_enc = pd.concat([X_train_num, X_train_cat], axis=1)
X_test_enc  = pd.concat([X_test_num, X_test_cat], axis=1)

In [12]:
X_train_enc[cat_col] = X_train_enc[cat_col].astype("category")
X_test_enc[cat_col] = X_test_enc[cat_col].astype("category")

### 【Light GBM + Optuna】

In [13]:
# ###################################################
# ############ Light GBM + Optuna####################
# ###################################################
# import optuna
# import lightgbm as lgb
# from sklearn.metrics import roc_auc_score
# from sklearn.model_selection import KFold
# from sklearn.model_selection import StratifiedKFold
# from sklearn.metrics import roc_auc_score

# import warnings
# warnings.filterwarnings("ignore")

# # 入力データ
# X = X_train
# y = y_train

# def objective(trial):

#     # パラメータ
#     lgbm_params = {
#         'objective': 'binary',
#         # "device": "cpu",
#         "device": "gpu",
#         'metric': 'auc',
#         'verbose': -1,              # ログ出力の制御
#         'boosting_type': 'gbdt',
#         'learning_rate': 0.1,
#         'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
#         'num_leaves': trial.suggest_int('num_leaves', 16, 256),
#         'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1),
#         'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
#         'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0,log=True),
#         'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0,log=True),
#         'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
#         }
    
#     oof_lgb = np.zeros(len(X))
#     fold_scores = []

#     # クロスバリデーション
#     skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
#     # for fold,(train_idx,valid_idx) in enumerate(kf.split(X)):
#     for fold,(train_idx,valid_idx) in enumerate(skf.split(X,y)):

#         print("#"*25)
#         print(f"### Fold {fold+1}")
#         print("#"*25)

    
#         X_train_kf = X.iloc[train_idx,:]
#         y_train_kf = y.iloc[train_idx]
#         X_valid_kf = X.iloc[valid_idx,:]
#         y_valid_kf = y.iloc[valid_idx]
    
#         # データセット作成
#         lgb_train_lgb = lgb.Dataset(X_train_kf,y_train_kf,
#                                     categorical_feature=cat_col)
#         lgb_valid_lgb = lgb.Dataset(X_valid_kf,y_valid_kf,
#                                     categorical_feature=cat_col)

#         # 学習
#         model_lgb = lgb.train(
#             lgbm_params,
#             lgb_train_lgb,
#             num_boost_round=1000,
#             valid_sets=[lgb_train_lgb,lgb_valid_lgb],
#             valid_names=["train","valid"],
#             callbacks=[
#                 lgb.early_stopping(stopping_rounds=50,verbose=False),
#                 lgb.log_evaluation(100),])
    
#         # 各foldでのバリデーション予測
#         oof_lgb[valid_idx] = model_lgb.predict(
#             X_valid_kf, num_iteration=model_lgb.best_iteration)

#         # AUCスコア算出
#         fold_scores.append(roc_auc_score(y_valid_kf,oof_lgb[valid_idx]))

#     score = np.mean(fold_scores)

#     return score
    
# study = optuna.create_study(direction="maximize",
#                            pruner=optuna.pruners.MedianPruner())
# study.optimize(objective, n_trials=5)

In [14]:
# for i, trial in enumerate(study.best_trials):
#   print(trial.params)
#   print([j for j in trial.values])

In [15]:
# print(study.best_trial)
# print(study.best_params)

In [16]:
# best_params = study.best_params

# best_params

### 【Light GBM】

In [17]:
###################################################
############ Light GBM ############################
###################################################
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

import warnings
warnings.filterwarnings("ignore")

# 学習、バリデーションデータ
pred_lgb = np.zeros(len(train_df))
pred_lgb_test = np.zeros(len(test_df))
models_lgb = []

# 入力データ
X = X_train_enc
y = y_train
X_test = X_test_enc

# 評価履歴を格納する辞書
evals_result_lgb = {}

# ベストパラメータ
lgbm_params = {
    'objective': 'binary',
    # "device": "cpu",
    "device": "gpu",
    'metric': 'auc',
    # 'metric': 'binary_logloss',
    'verbose': -1,              # ログ出力の制御
    'boosting_type': 'gbdt',
    'learning_rate': 0.03,
    'num_leaves': 63,            # 31〜255の中核
    'feature_fraction': 0.8,     # = colsample_bytree
    'bagging_fraction': 0.8,     # = subsample
    'bagging_freq': 5,
    'min_data_in_leaf': 100,     # 50〜300で効く
    'lambda_l1': 1e-3,
    'lambda_l2': 1e-2,
    'max_bin': 255,              # GPUで十分
    'verbosity': -1 
}

# クロスバリデーション
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold,(train_idx,valid_idx) in enumerate(skf.split(X,y)):

    print("#"*25)
    print(f"### Fold {fold+1}")
    print("#"*25)
    
    X_train_kf = X.iloc[train_idx,:]
    y_train_kf = y.iloc[train_idx]
    X_valid_kf = X.iloc[valid_idx,:]
    y_valid_kf = y.iloc[valid_idx]
    
    # データセット作成
    lgb_train_lgb = lgb.Dataset(X_train_kf,y_train_kf,
                                categorical_feature=cat_col)
    lgb_valid_lgb = lgb.Dataset(X_valid_kf,y_valid_kf,
                                categorical_feature=cat_col)

    # lgb_train_lgb = lgb.Dataset(X_train_kf,y_train_kf)
    # lgb_valid_lgb = lgb.Dataset(X_valid_kf,y_valid_kf)

    # 学習
    model_lgb = lgb.train(
        lgbm_params,
        lgb_train_lgb,
        # num_boost_round=1,
        num_boost_round=2000,
        valid_sets=[lgb_train_lgb,lgb_valid_lgb],
        valid_names=["train","valid"],
        callbacks=[
            lgb.early_stopping(stopping_rounds=100,verbose=False),
            lgb.record_evaluation(evals_result_lgb),
            lgb.log_evaluation(100),])

    # 各foldでのバリデーション予測
    pred_lgb[valid_idx] = model_lgb.predict(
        X_valid_kf, num_iteration=model_lgb.best_iteration)
    
    # モデルの追加
    models_lgb.append(model_lgb)

    # テストの予測
    # pred_lgb_test = pred_lgb_test + model_lgb.predict(
    #     X_test, num_iteration=model_lgb.best_iteration)


for model in models_lgb:
    pred_lgb_test = pred_lgb_test + model.predict(
        X_test, num_iteration=model.best_iteration)

# FOLD数で割る
pred_lgb_test = pred_lgb_test/5

#########################
### Fold 1
#########################


1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


[100]	train's auc: 0.963304	valid's auc: 0.963054
[200]	train's auc: 0.967712	valid's auc: 0.966904
[300]	train's auc: 0.970265	valid's auc: 0.968715
[400]	train's auc: 0.971906	valid's auc: 0.969561
[500]	train's auc: 0.973147	valid's auc: 0.970023
[600]	train's auc: 0.974282	valid's auc: 0.9704
[700]	train's auc: 0.975351	valid's auc: 0.970773
[800]	train's auc: 0.976288	valid's auc: 0.971051
[900]	train's auc: 0.977135	valid's auc: 0.971186
[1000]	train's auc: 0.977884	valid's auc: 0.971355
[1100]	train's auc: 0.978663	valid's auc: 0.971485
[1200]	train's auc: 0.979329	valid's auc: 0.971528
[1300]	train's auc: 0.979971	valid's auc: 0.971603
[1400]	train's auc: 0.98064	valid's auc: 0.971678
[1500]	train's auc: 0.981262	valid's auc: 0.971765
[1600]	train's auc: 0.981872	valid's auc: 0.971825
[1700]	train's auc: 0.982451	valid's auc: 0.971857
[1800]	train's auc: 0.982991	valid's auc: 0.971914
[1900]	train's auc: 0.983519	valid's auc: 0.971936
[2000]	train's auc: 0.984042	valid's auc: 0

In [18]:
# 訓練データのスコア
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt

AUC_lgb = roc_auc_score(y,pred_lgb)
F1_lgb = f1_score(y,np.round(pred_lgb,0))
print(f"LGB: AUC score = {AUC_lgb}, F1 = {F1_lgb}")

# 学習曲線
# lgb.plot_metric(evals_result_lgb,title="LightGBM AUC",)

LGB: AUC score = 0.9712509918468701, F1 = 0.740635361201275


In [19]:
# import shap
# import lightgbm as lgb
# import matplotlib.pyplot as plt
# shap.initjs()

# # サンプリング
# X_sample = X.sample(2000, random_state=0)

# shapval = 0
# for i in range(5):
#     explainer = shap.TreeExplainer(models_lgb[i])
#     shap_values = explainer.shap_values(X_sample)
#     if isinstance(shap_values, list):  # 2クラス分類
#         shap_values = shap_values[1]
#     shapval += shap_values

# shap.summary_plot(shapval, X_sample, plot_type="bar")

In [20]:
# # 重要度の算出
# feature_importances = 0
# for i in range(5):
#   feature_importances += pd.DataFrame(models_lgb[i].feature_importance(),columns=['importance'])

# feature=pd.DataFrame(X.columns,columns=['feature'])
# o=pd.concat([feature,feature_importances],axis=1)
# o = o.sort_values("importance", ascending=False)
# print(o)

In [21]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# plt.figure(figsize=(16, 16))
# sns.barplot(data=o.sort_values(by='importance', ascending=False), x='importance', y='feature')
# plt.title('Feature Importances ')

In [22]:
# # 提出データ作成
# sample_submission = pd.read_csv("/kaggle/input/playground-series-s5e8/sample_submission.csv")

# sample_submission['y'] = pred_lgb_test
# sample_submission.to_csv('submission.csv', index=False)
# print('Submission file saved.')

### 【XGBoost + Optuna】

In [23]:
# #################################################
# ############ XGBoost ############################
# #################################################
# import optuna
# import xgboost as xgb
# from sklearn.metrics import roc_auc_score
# from sklearn.model_selection import KFold
# from sklearn.model_selection import StratifiedKFold
# from sklearn.metrics import roc_auc_score

# # 学習、バリデーションデータ
# pred_xgb = np.zeros(len(train_df))
# pred_xgb_test = np.zeros(len(test_df))
# models_xgb = []

# # 入力データ
# X = X_train
# y = y_train

# # --- fold ごとに DMatrix を事前作成 ---
# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# dtrain_folds = []
# dvalid_folds = []

# for train_idx, valid_idx in skf.split(X, y):
#     X_train_kf, X_valid_kf = X.iloc[train_idx], X.iloc[valid_idx]
#     y_train_kf, y_valid_kf = y.iloc[train_idx], y.iloc[valid_idx]

#     dtrain = xgb.DMatrix(X_train_kf, label=y_train_kf, enable_categorical=True)
#     dvalid = xgb.DMatrix(X_valid_kf, label=y_valid_kf, enable_categorical=True)

#     dtrain_folds.append(dtrain)
#     dvalid_folds.append(dvalid)

# # --- Optuna objective ---
# def objective(trial):
#     params = {
#         "objective": "binary:logistic",
#         "tree_method": "hist",  # GPU 用
#         "device": "cuda",
#         "eval_metric": "auc",
#         "booster": trial.suggest_categorical("booster", ["gbtree", "dart"]),
#         "lambda": trial.suggest_float("lambda", 1e-8, 10.0, log=True),
#         "alpha": trial.suggest_float("alpha", 1e-8, 10.0, log=True),
#         "colsample_bytree": trial.suggest_float("colsample_bytree", 0.3, 1.0),
#         "subsample": trial.suggest_float("subsample", 0.3, 1.0),
#         "max_depth": trial.suggest_int("max_depth", 3, 10),
#         "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
#         "gamma": trial.suggest_float("gamma", 1e-8, 10.0, log=True),
#         "learning_rate": 0.1
#     }

#     scores = []

#     # trial 内では DMatrix を再利用
#     for dtrain, dvalid in zip(dtrain_folds, dvalid_folds):
#         model = xgb.train(
#             params,
#             dtrain,
#             num_boost_round=1000,
#             evals=[(dtrain, "train"), (dvalid, "valid")],
#             early_stopping_rounds=50,
#             verbose_eval=100
#         )

#         preds = model.predict(dvalid, iteration_range=(0, model.best_iteration+1))
#         scores.append(roc_auc_score(dvalid.get_label(), preds))

#     return np.mean(scores)

# # --- Optuna 実行 ---
# study = optuna.create_study(direction="maximize",
#                            pruner=optuna.pruners.MedianPruner())
# study.optimize(objective, n_trials=5)  # 仮チューニング用に少なめ

# print("Best Score:", study.best_value)
# print("Best Params:", study.best_params)

### 【XGBoost】

In [24]:
#################################################
############ XGBoost ############################
#################################################
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 学習、バリデーションデータ
pred_xgb = np.zeros(len(train_df))
pred_xgb_test = np.zeros(len(test_df))
models_xgb = []

# 入力データ
X = X_train_enc
y = y_train
X_test = X_test_enc


# 評価履歴を保存する辞書
evals_result_xgb = {}

# パラメータ
xgb_params = {
    "objective": "binary:logistic",  
    "eval_metric": "auc",           
    "learning_rate": 0.1,
    "max_depth": 0,
    "subsample": 0.8,
    "colsample_bytree": 0.7,
    "tree_method": "gpu_hist",
    "gpu_id": 0,
    "grow_policy": "lossguide", 
    "max_leaves": 32,           
    "alpha": 2.0,
}

# クロスバリデーション
# kf = KFold(n_splits=5, shuffle=True, random_state=42)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# for fold, (train_idx,valid_idx) in enumerate(kf.split(X)):
for fold,(train_idx,valid_idx) in enumerate(skf.split(X,y)):

    print("#"*25)
    print(f"### Fold {fold+1}")
    print("#"*25)

    # foldごとの訓練、バリデーションデータ
    X_train_kf = X.iloc[train_idx,:]
    y_train_kf = y.iloc[train_idx]
    X_valid_kf = X.iloc[valid_idx,:]
    y_valid_kf = y.iloc[valid_idx]

    # DMatrixに変換
    dtrain = xgb.DMatrix(X_train_kf,label=y_train_kf,enable_categorical=True)
    dvalid = xgb.DMatrix(X_valid_kf,label=y_valid_kf,enable_categorical=True)

    # 学習
    model_xgb = xgb.train(
        xgb_params,
        dtrain,
        # num_boost_round=1,
        num_boost_round=2000,
        evals=[(dtrain,"train"),(dvalid,"valid")],
        early_stopping_rounds=100,
        evals_result=evals_result_xgb,
        verbose_eval=100,
    )

    # 各foldでのバリデーション予測
    pred_xgb[valid_idx] = model_xgb.predict(
        dvalid,
        iteration_range=(0,model_xgb.best_iteration+1))

    # モデルの追加
    models_xgb.append(model_xgb)

# テストの予測
dtest = xgb.DMatrix(X_test,enable_categorical=True)

for model in models_xgb:
    pred_xgb_test = pred_xgb_test + model.predict(
        dtest,
        iteration_range=(0,model.best_iteration+1))

# FOLD数で割る
pred_xgb_test = pred_xgb_test/5

#########################
### Fold 1
#########################
[0]	train-auc:0.94041	valid-auc:0.94143
[100]	train-auc:0.96711	valid-auc:0.96684
[200]	train-auc:0.97057	valid-auc:0.96930
[300]	train-auc:0.97251	valid-auc:0.97014
[400]	train-auc:0.97401	valid-auc:0.97062
[500]	train-auc:0.97530	valid-auc:0.97093
[600]	train-auc:0.97648	valid-auc:0.97124
[700]	train-auc:0.97754	valid-auc:0.97143
[800]	train-auc:0.97851	valid-auc:0.97159
[900]	train-auc:0.97940	valid-auc:0.97168
[1000]	train-auc:0.98024	valid-auc:0.97177
[1100]	train-auc:0.98105	valid-auc:0.97184
[1200]	train-auc:0.98180	valid-auc:0.97189
[1300]	train-auc:0.98252	valid-auc:0.97195
[1400]	train-auc:0.98317	valid-auc:0.97196
[1475]	train-auc:0.98368	valid-auc:0.97194
#########################
### Fold 2
#########################
[0]	train-auc:0.94041	valid-auc:0.94002
[100]	train-auc:0.96726	valid-auc:0.96554
[200]	train-auc:0.97077	valid-auc:0.96820
[300]	train-auc:0.97270	valid-auc:0.96914
[400]	train-auc:0.97423	valid-au

In [25]:
# 訓練データのスコア
from sklearn.metrics import f1_score

AUC_xgb = roc_auc_score(y,pred_xgb)
F1_xgb = f1_score(y,np.round(pred_xgb,0))
print(f"XGB: AUC score = {AUC_xgb}, F1 = {F1_xgb}")

XGB: AUC score = 0.9711630910732361, F1 = 0.7396576153864021


In [26]:
# # 提出データ作成
# sample_submission = pd.read_csv("/kaggle/input/playground-series-s5e8/sample_submission.csv")

# sample_submission['y'] = pred_xgb_test
# sample_submission.to_csv('submission.csv', index=False)
# print('Submission file saved.')

In [27]:
# # 学習曲線の描画
# from matplotlib.pyplot as plt
# epochs = len(evals_result_xgb['train']['auc'])
# x_axis = range(0, epochs)

# plt.figure()
# plt.plot(x_axis, evals_result_xgb['train']['auc'], label='Train')
# plt.plot(x_axis, evals_result_xgb['valid']['auc'], label='Validation')
# plt.xlabel('Iteration')
# plt.ylabel('AUC')
# plt.title('XGBoost AUC')
# plt.grid()
# plt.legend()
# plt.show()

### 【CatBoost + Optuna】

In [28]:
# #################################################
# ############ CatBoost ############################
# #################################################
# # from catboost import Pool, train
# from catboost import CatBoostClassifier, Pool
# from sklearn.metrics import roc_auc_score
# from sklearn.model_selection import StratifiedKFold
# import optuna
# import numpy as np

# # 入力データ
# X = X_train
# y = y_train

# # 履歴を保存
# cb_auc_valid = []


# def objective(trial):

#     cat_params = {
#         'iterations' : trial.suggest_int('iterations', 300, 2000),
#         'depth' : trial.suggest_int('depth', 6, 10),
#         'random_strength' :trial.suggest_int('random_strength', 1, 20),
#         'bagging_temperature' :trial.suggest_float('bagging_temperature', 0.1, 10.0, log=True),
#         'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
#         'od_wait' :trial.suggest_int('od_wait', 10, 50),
#         'learning_rate' : trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
#         "loss_function": "Logloss",
#         "eval_metric": "AUC",
#         # "task_type": "CPU",
#         # 'learning_rate' : 0.1,
#         "task_type": "GPU",
#         # "devices": "0",
#         "verbose": False,
#     }
    
#     skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
#     scores = []
    
#     for fold, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
#         print(f"  >> Fold {fold+1}/{skf.n_splits} 開始")
#         X_train_kf, y_train_kf = X.iloc[train_idx].copy(), y.iloc[train_idx]
#         X_valid_kf, y_valid_kf = X.iloc[valid_idx].copy(), y.iloc[valid_idx]

#         # カテゴリ変換
#         for col in cat_col:
#             X_train_kf[col] = X_train_kf[col].astype("category")
#             X_valid_kf[col] = X_valid_kf[col].astype("category")

#         train_pool = Pool(X_train_kf, y_train_kf, cat_features=cat_col)
#         valid_pool = Pool(X_valid_kf, y_valid_kf, cat_features=cat_col)

#         # 学習
#         model_cb = CatBoostClassifier(**cat_params)
#         model_cb.fit(train_pool, eval_set=valid_pool, early_stopping_rounds=50)

#         preds = model_cb.predict_proba(X_valid_kf)[:,1]
#         score = roc_auc_score(y_valid_kf, preds)
#         scores.append(score)

#     return np.mean(scores)

# # --- Optuna 実行 ---
# study = optuna.create_study(direction="maximize",
#                             pruner=optuna.pruners.MedianPruner())
# study.optimize(objective, n_trials=5)

# print("Best Score:", study.best_value)
# print("Best Params:", study.best_params)

### 【CatBoost】

In [29]:
#################################################
############ CatBoost (Classifier版) ############
#################################################
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import StratifiedKFold
import numpy as np

# 学習、バリデーションデータ
pred_cb = np.zeros(len(train_df))
pred_cb_test = np.zeros(len(test_df))
models_cb = []
cb_auc_valid = []  # foldごとのAUC履歴

# 入力データ
X = X_train_enc
y = y_train
X_test = X_test_enc

# CatBoostパラメータ
cat_params = {
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "depth": 8,                   # 6〜10
    "learning_rate": 0.05,        # 0.03〜0.1
    # "iterations": 1,          # 大きめ＋ES
    "iterations": 2000,          # 大きめ＋ES
    "bootstrap_type": "Bayesian", # 精度安定
    "boosting_type": "Ordered",   # 多カテゴリに強い
    "random_strength": 1.0,       # 0.5〜2.0で微調整
    "task_type": "GPU",
    # "task_type": "CPU",           # このデータ規模ならCPUの方が速い/安定なこと多い
    "verbose": 100,
}

# Stratified KFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, valid_idx) in enumerate(skf.split(X, y)):

    print("#" * 25)
    print(f"### Fold {fold+1}")
    print("#" * 25)

    # データ分割
    X_train_kf = X.iloc[train_idx, :]
    y_train_kf = y.iloc[train_idx]
    X_valid_kf = X.iloc[valid_idx, :]
    y_valid_kf = y.iloc[valid_idx]

    # object型をカテゴリ型に変換
    for col in cat_col:
        X_train_kf.loc[:, col] = X_train_kf.loc[:, col].astype("category")
        X_valid_kf.loc[:, col] = X_valid_kf.loc[:, col].astype("category")

    # Poolを作成
    train_pool = Pool(X_train_kf, y_train_kf, cat_features=cat_col)
    valid_pool = Pool(X_valid_kf, y_valid_kf, cat_features=cat_col)

    # モデル作成 & 学習
    model_cb = CatBoostClassifier(**cat_params)
    model_cb.fit(
        train_pool,
        eval_set=valid_pool,
        early_stopping_rounds=100,
        use_best_model=True
    )

    # バリデーション予測
    pred_cb[valid_idx] = model_cb.predict_proba(X_valid_kf)[:, 1]

    # モデル保存
    models_cb.append(model_cb)

    # foldごとのベストスコアを保存
    cb_auc_valid.append(model_cb.get_best_score()["validation"]["AUC"])

# テスト予測
for model in models_cb:
    pred_cb_test += model.predict_proba(X_test)[:, 1]

# FOLD数で割って平均化
pred_cb_test = pred_cb_test / skf.n_splits

print("各foldのAUC:", cb_auc_valid)
print("平均AUC:", np.mean(cb_auc_valid))

#########################
### Fold 1
#########################


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.9263818	best: 0.9263818 (0)	total: 12.1s	remaining: 6h 43m 32s
100:	test: 0.9606615	best: 0.9606615 (100)	total: 38.3s	remaining: 11m 59s
200:	test: 0.9639385	best: 0.9639385 (200)	total: 1m 4s	remaining: 9m 34s
300:	test: 0.9654797	best: 0.9654797 (300)	total: 1m 29s	remaining: 8m 27s
400:	test: 0.9665264	best: 0.9665264 (400)	total: 1m 55s	remaining: 7m 40s
500:	test: 0.9671269	best: 0.9671269 (500)	total: 2m 21s	remaining: 7m 2s
600:	test: 0.9675694	best: 0.9675703 (597)	total: 2m 46s	remaining: 6m 28s
700:	test: 0.9677940	best: 0.9677940 (700)	total: 3m 12s	remaining: 5m 56s
800:	test: 0.9680870	best: 0.9680870 (800)	total: 3m 37s	remaining: 5m 25s
900:	test: 0.9683189	best: 0.9683189 (900)	total: 4m 2s	remaining: 4m 55s
1000:	test: 0.9684880	best: 0.9684884 (998)	total: 4m 27s	remaining: 4m 26s
1100:	test: 0.9686474	best: 0.9686474 (1100)	total: 4m 51s	remaining: 3m 57s
1200:	test: 0.9688209	best: 0.9688209 (1200)	total: 5m 16s	remaining: 3m 30s
1300:	test: 0.9689328	be

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.9334250	best: 0.9334250 (0)	total: 256ms	remaining: 8m 32s
100:	test: 0.9595940	best: 0.9595940 (100)	total: 26.5s	remaining: 8m 17s
200:	test: 0.9627956	best: 0.9627956 (200)	total: 52.4s	remaining: 7m 49s
300:	test: 0.9642831	best: 0.9642831 (300)	total: 1m 18s	remaining: 7m 21s
400:	test: 0.9653062	best: 0.9653062 (400)	total: 1m 43s	remaining: 6m 53s
500:	test: 0.9659204	best: 0.9659204 (500)	total: 2m 9s	remaining: 6m 26s
600:	test: 0.9664220	best: 0.9664220 (599)	total: 2m 34s	remaining: 6m
700:	test: 0.9667678	best: 0.9667678 (700)	total: 3m	remaining: 5m 34s
800:	test: 0.9670483	best: 0.9670483 (800)	total: 3m 25s	remaining: 5m 8s
900:	test: 0.9672510	best: 0.9672513 (899)	total: 3m 51s	remaining: 4m 42s
1000:	test: 0.9674456	best: 0.9674456 (1000)	total: 4m 16s	remaining: 4m 15s
1100:	test: 0.9676343	best: 0.9676343 (1100)	total: 4m 42s	remaining: 3m 50s
1200:	test: 0.9677685	best: 0.9677685 (1200)	total: 5m 7s	remaining: 3m 24s
1300:	test: 0.9678881	best: 0.9678881

...

0:	test: 0.9322157	best: 0.9322157 (0)	total: 257ms	remaining: 8m 34s
100:	test: 0.9592852	best: 0.9592852 (100)	total: 26.5s	remaining: 8m 17s
200:	test: 0.9626350	best: 0.9626350 (200)	total: 52.4s	remaining: 7m 48s
300:	test: 0.9644019	best: 0.9644019 (300)	total: 1m 18s	remaining: 7m 21s
400:	test: 0.9654436	best: 0.9654436 (400)	total: 1m 43s	remaining: 6m 54s
500:	test: 0.9660925	best: 0.9660925 (500)	total: 2m 9s	remaining: 6m 27s
600:	test: 0.9665928	best: 0.9665928 (600)	total: 2m 34s	remaining: 6m
700:	test: 0.9669468	best: 0.9669468 (700)	total: 3m	remaining: 5m 34s
800:	test: 0.9671978	best: 0.9671979 (799)	total: 3m 25s	remaining: 5m 7s
900:	test: 0.9673776	best: 0.9673776 (898)	total: 3m 50s	remaining: 4m 41s
1000:	test: 0.9675514	best: 0.9675514 (1000)	total: 4m 16s	remaining: 4m 15s
1100:	test: 0.9676906	best: 0.9676906 (1100)	total: 4m 41s	remaining: 3m 49s
1200:	test: 0.9678118	best: 0.9678118 (1200)	total: 5m 6s	remaining: 3m 23s
1300:	test: 0.9679107	best: 0.9679107

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.9260441	best: 0.9260441 (0)	total: 255ms	remaining: 8m 30s
100:	test: 0.9603126	best: 0.9603126 (100)	total: 26.4s	remaining: 8m 16s
200:	test: 0.9635664	best: 0.9635664 (200)	total: 52.4s	remaining: 7m 49s
300:	test: 0.9650748	best: 0.9650748 (300)	total: 1m 18s	remaining: 7m 21s
400:	test: 0.9660447	best: 0.9660447 (400)	total: 1m 43s	remaining: 6m 53s
500:	test: 0.9667212	best: 0.9667212 (500)	total: 2m 9s	remaining: 6m 26s
600:	test: 0.9672194	best: 0.9672194 (600)	total: 2m 34s	remaining: 6m
700:	test: 0.9675728	best: 0.9675728 (700)	total: 3m	remaining: 5m 33s
800:	test: 0.9678580	best: 0.9678580 (800)	total: 3m 25s	remaining: 5m 7s
900:	test: 0.9681032	best: 0.9681032 (900)	total: 3m 51s	remaining: 4m 42s
1000:	test: 0.9682992	best: 0.9682992 (1000)	total: 4m 16s	remaining: 4m 15s
1100:	test: 0.9685537	best: 0.9685537 (1100)	total: 4m 41s	remaining: 3m 50s
1200:	test: 0.9686942	best: 0.9686942 (1200)	total: 5m 6s	remaining: 3m 24s
1300:	test: 0.9688292	best: 0.9688292

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.9328085	best: 0.9328085 (0)	total: 259ms	remaining: 8m 37s
100:	test: 0.9596044	best: 0.9596044 (100)	total: 26.5s	remaining: 8m 17s
200:	test: 0.9629774	best: 0.9629774 (200)	total: 52.4s	remaining: 7m 49s
300:	test: 0.9647277	best: 0.9647277 (300)	total: 1m 18s	remaining: 7m 21s
400:	test: 0.9657197	best: 0.9657197 (400)	total: 1m 43s	remaining: 6m 52s
500:	test: 0.9663556	best: 0.9663556 (500)	total: 2m 9s	remaining: 6m 26s
600:	test: 0.9668278	best: 0.9668278 (599)	total: 2m 34s	remaining: 6m
700:	test: 0.9671682	best: 0.9671682 (700)	total: 3m	remaining: 5m 33s
800:	test: 0.9675022	best: 0.9675022 (800)	total: 3m 25s	remaining: 5m 7s
900:	test: 0.9677370	best: 0.9677370 (900)	total: 3m 51s	remaining: 4m 41s
1000:	test: 0.9679247	best: 0.9679247 (1000)	total: 4m 16s	remaining: 4m 15s
1100:	test: 0.9681212	best: 0.9681212 (1100)	total: 4m 42s	remaining: 3m 50s
1200:	test: 0.9682399	best: 0.9682399 (1199)	total: 5m 7s	remaining: 3m 24s
1300:	test: 0.9683238	best: 0.9683238

In [30]:
from sklearn.metrics import f1_score

# スコア表示
AUC_cb = roc_auc_score(y,pred_cb)
F1_cb = f1_score(y,np.round(pred_cb,0))
print(f"CB: AUC score = {AUC_cb}, F1 = {F1_cb}")

# # 学習履歴を一番短いfoldに揃える
# min_len = min(len(m) for m in cb_auc_valid)
# cb_auc_score = [m[:min_len] for m in cb_auc_valid]

# # foldごとの結果を平均する
# cb_auc_score = np.average(cb_auc_score,axis=0)

CB: AUC score = 0.9688984460370684, F1 = 0.7249669530733641


In [31]:
# # 提出データ作成
# sample_submission = pd.read_csv("/kaggle/input/playground-series-s5e8/sample_submission.csv")

# sample_submission['y'] = pred_cb_test
# sample_submission.to_csv('submission.csv', index=False)
# print('Submission file saved.')

In [32]:
# import  matplotlib.pyplot as plt
# # 履歴の可視化
# plt.plot(cb_auc_score, label='Validation')
# plt.xlabel('Iteration')
# plt.ylabel('AUC')
# plt.grid()
# plt.legend()
# plt.title("CabBoost AUC")
# plt.show()

### Stacking

In [33]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import roc_auc_score

print("# STACKING ENSEMBLE IMPLEMENTATION")
print("# " + "="*50)
print("# Combining top 3 models: LightGBM, XGBoost, CatBoost")
print("# Meta-learner: Logistic Regression")
print("# " + "="*50)

stacking_train = pd.DataFrame({
    'lgb': pred_lgb,
    'xgb': pred_xgb, 
    'cat': pred_cb
})

stacking_test = pd.DataFrame({
    'lgb': pred_lgb_test,
    'xgb': pred_xgb_test,
    'cat': pred_cb_test
})

print(f"# Stacking train shape: {stacking_train.shape}")
print(f"# Stacking test shape: {stacking_test.shape}")

print("\n# METHOD 1: WEIGHTED AVERAGE")
print("# " + "-"*30)

scores = [AUC_lgb, AUC_xgb, AUC_cb]  
total_score = sum(scores)
weights = [score/total_score for score in scores]

print(f"# Model weights:")
print(f"# LightGBM: {weights[0]:.4f}")
print(f"# XGBoost:  {weights[1]:.4f}")
print(f"# CatBoost: {weights[2]:.4f}")

weighted_oof = (stacking_train['lgb'] * weights[0] + 
                stacking_train['xgb'] * weights[1] + 
                stacking_train['cat'] * weights[2])

weighted_test = (stacking_test['lgb'] * weights[0] + 
                 stacking_test['xgb'] * weights[1] + 
                 stacking_test['cat'] * weights[2])

weighted_score = roc_auc_score(y, weighted_oof)
print(f"# Weighted Average ROC AUC: {weighted_score:.6f}")

print("\n# METHOD 2: LOGISTIC REGRESSION META-LEARNER")
print("# " + "-"*40)

meta_learner = LogisticRegression(random_state=42, max_iter=1000)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

cv_scores = cross_val_score(meta_learner, stacking_train, y, 
                           cv=skf, scoring='roc_auc', n_jobs=-1)

print(f"# Meta-learner CV scores: {[f'{score:.6f}' for score in cv_scores]}")
print(f"# Meta-learner mean CV: {cv_scores.mean():.6f} ± {cv_scores.std():.6f}")

meta_learner.fit(stacking_train, y)
meta_oof = meta_learner.predict_proba(stacking_train)[:, 1]
meta_test = meta_learner.predict_proba(stacking_test)[:, 1]
meta_score = roc_auc_score(y, meta_oof)

print(f"# Meta-learner ROC AUC: {meta_score:.6f}")

coefficients = meta_learner.coef_[0]
print(f"# Meta-learner coefficients:")
print(f"# LightGBM: {coefficients[0]:.4f}")
print(f"# XGBoost:  {coefficients[1]:.4f}")
print(f"# CatBoost: {coefficients[2]:.4f}")
print(f"# Intercept: {meta_learner.intercept_[0]:.4f}")

print("\n# METHOD 3: SIMPLE AVERAGE (BASELINE)")
print("# " + "-"*35)

simple_oof = (stacking_train['lgb'] + stacking_train['xgb'] + stacking_train['cat']) / 3
simple_test = (stacking_test['lgb'] + stacking_test['xgb'] + stacking_test['cat']) / 3
simple_score = roc_auc_score(y, simple_oof)

print(f"# Simple Average ROC AUC: {simple_score:.6f}")

print("\n# ENSEMBLE METHODS COMPARISON")
print("# " + "="*40)
ensemble_results = [
    ('Individual LightGBM', AUC_lgb),
    ('Individual XGBoost', AUC_xgb),
    ('Individual CatBoost', AUC_cb),
    ('Weighted Average', weighted_score),
    ('Meta-learner (LogReg)', meta_score),
    ('Simple Average', simple_score)
]

ensemble_results.sort(key=lambda x: x[1], reverse=True)

for i, (method, score) in enumerate(ensemble_results, 1):
    print(f"# {i}. {method:<25}: {score:.6f}")

best_method, best_score = ensemble_results[0]
print(f"\n# BEST ENSEMBLE METHOD: {best_method}")
print(f"# BEST ENSEMBLE SCORE: {best_score:.6f}")

if 'Meta-learner' in best_method:
    final_oof = meta_oof
    final_test = meta_test
    print("# Using Meta-learner predictions for final submission")
elif 'Weighted' in best_method:
    final_oof = weighted_oof
    final_test = weighted_test
    print("# Using Weighted Average predictions for final submission")
else:
    final_oof = simple_oof
    final_test = simple_test
    print("# Using Simple Average predictions for final submission")

print("\n# STACKING ENSEMBLE COMPLETED!")
print("# " + "="*50)

# STACKING ENSEMBLE IMPLEMENTATION
# ==================================================
# Combining top 3 models: LightGBM, XGBoost, CatBoost
# Meta-learner: Logistic Regression
# ==================================================
# Stacking train shape: (750000, 3)
# Stacking test shape: (250000, 3)

# METHOD 1: WEIGHTED AVERAGE
# ------------------------------
# Model weights:
# LightGBM: 0.3336
# XGBoost:  0.3336
# CatBoost: 0.3328
# Weighted Average ROC AUC: 0.971182

# METHOD 2: LOGISTIC REGRESSION META-LEARNER
# ----------------------------------------
# Meta-learner CV scores: ['0.972209', '0.971158', '0.970860', '0.971839', '0.971289']
# Meta-learner mean CV: 0.971471 ± 0.000487
# Meta-learner ROC AUC: 0.971473
# Meta-learner coefficients:
# LightGBM: 3.9558
# XGBoost:  2.9199
# CatBoost: 0.9000
# Intercept: -4.0619

# METHOD 3: SIMPLE AVERAGE (BASELINE)
# -----------------------------------
# Simple Average ROC AUC: 0.971181

# ENSEMBLE METHODS COMPARISON
# ===================

In [34]:
# 提出データ作成
sample_submission = pd.read_csv("/kaggle/input/playground-series-s5e8/sample_submission.csv")

sample_submission['y'] = final_test
sample_submission.to_csv('submission.csv', index=False)
print('Submission file saved.')

Submission file saved.


### 【Neural Net】

In [35]:
# import random
# import os
# import pandas as pd
# import numpy as np
# from tqdm.notebook import tqdm
# import matplotlib.pyplot as plt

# import torch
# import torch.nn as nn
# from torch.utils.data import Dataset, DataLoader

# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import f1_score

# # pytorch実装
# import torch # Tensorの作成や操作
# import torch.nn as nn # ニューラルネットワーク
# import torch.nn.functional as F # 関数をメソッドとして提供
# import torch.optim as optim # オプティマイザ
# from torch.utils.data import Dataset, DataLoader
# from torch.autograd import Variable

# from sklearn.model_selection import StratifiedKFold
# from sklearn.metrics import roc_auc_score
# from tqdm.notebook import tqdm
# # from tqdm import tqdm
# import matplotlib.pyplot as plt
# import time

# # GPUの使用状況確認
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(device)

# all_df = all_df_NN
# all_df = all_df.drop(["id","y"],axis=1)
# y = train_df["y"]

# # 設定
# SEED = 42
# TARGET = "y"

# CATEGORICAL = cat_col
# NUMERICAL = num_col
# USE = CATEGORICAL + NUMERICAL
# # df_train = train_df.drop("id",axis=1)
# # df_test = test_df.drop("id",axis=1)

In [36]:
# # (1) 住宅ローン + ローン
# all_df["housing_loan"] = all_df["housing"].astype(str) + "_" + all_df["loan"].astype(str)

# # (2) コンタクト時間 x 年齢
# all_df["duration_x_age"] = all_df["duration"] * all_df["age"]

# # (3) sin,cos(コンタクト時間)
# all_df['duration_sin'] = np.sin(2*np.pi * all_df['duration'] / 400)
# all_df['duration_cos'] = np.cos(2*np.pi * all_df['duration'] / 400)

# # (4) monthを数値に直し周期的に使う
# month_map = {'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4,
#     'may': 5, 'jun': 6, 'jul': 7, 'aug': 8,
#     'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12}
# all_df['month_num'] = all_df['month'].map(month_map).astype('int')
# all_df['month_sin'] = np.sin(2 * np.pi * all_df['month_num'] / 12)
# all_df['month_cos'] = np.cos(2 * np.pi * all_df['month_num'] / 12)

# # (5) コンタクト時間をカテゴリ化
# all_df['duration_bin'] = pd.cut(
#     all_df['duration'],
#     bins=[0, 60, 300, 600, 900, float('inf')],
#     labels=['short', 'medium', 'long', 'very_long', 'extreme'],
#     right=False)
# all_df['duration_bin'] = all_df['duration_bin'].astype("object")

# # (6) 連絡手段 + 年齢
# all_df['age_group'] = pd.cut(
#     all_df['age'],
#     bins=[0, 30, 45, 60, 100],
#     labels=['young', 'mid', 'senior', 'elder'])
# all_df["contact_age"] = all_df["contact"].astype(str) + "_" + all_df["age_group"].astype(str)
# all_df = all_df.drop("age_group",axis=1)

# # (7) sin,cos(pdays)
# all_df['pdays_sin'] = np.sin(2*np.pi * all_df['pdays'] / 90)
# all_df['pdays_cos'] = np.cos(2*np.pi * all_df['pdays'] / 90)

In [37]:
# # 標準化 + ラベルエンコード
# def preprocessing(all_df, cat_cols=CATEGORICAL, num_cols=NUMERICAL, target=TARGET):

#     # 訓練データとテストデータに分離
#     train = all_df[:len(train_df)]
#     test = all_df[len(train_df):]

#     # y = train[target]
#     # train = train.drop("y",axis=1)
#     train_len = len(train)

#     # 訓練データ + テストデータ
#     # df = pd.concat([train.drop(columns=target), test])
#     # y = train[target]
#     # train_len = len(train)
    
#     # 欠損埋め
#     # df[cat_cols] = df[cat_cols].fillna('None')
#     # df[num_cols] = df[num_cols].fillna(0)

#     # train = df[:train_len]
#     # test = df[train_len:]

#     # 標準化
#     scaler = StandardScaler()

#     # フィッティング
#     # scaler.fit(df[num_cols])
#     scaler.fit(train[num_cols])

#     # 適用
#     train[num_cols] = scaler.transform(train[num_cols])
#     test[num_cols] = scaler.transform(test[num_cols])
#     df = pd.concat([train, test])
    
#     # ラベルエンコーダ
#     for col in df.columns:
#         if col in cat_cols:
#             df[col] = LabelEncoder().fit_transform(df[col])
#             df[col]= df[col].astype('category')
            
#     return pd.concat([df.iloc[:train_len], y], axis=1), df.iloc[train_len:]

In [38]:
# # 前処理の実施
# df_train, df_test = preprocessing(all_df)

In [39]:
# # データセット関数
# class CustomDataset(Dataset):

#     # オブジェクト定義
#     def __init__(self, df, target, cat_cols=CATEGORICAL):
#         self.df_cat = df[cat_cols]
#         self.df_num = df.drop(cat_cols, axis=1)
#         self.X_cats = self.df_cat.values.astype(np.int64)
#         self.X_nums = self.df_num.values.astype(np.float32)
#         self.target = target.values.astype(np.int64)

#     # データセットのサイズを返す
#     def __len__(self):
#         return len(self.target)

#     # 指定したインデックスのデータとラベルを返す
#     def __getitem__(self, idx):
#         return [self.X_cats[idx], self.X_nums[idx], self.target[idx]]

In [40]:
# # NNモデル作成
# class NN_Model(nn.Module):

#     # ネットワーク構造の定義
#     def __init__(self, embedding_sizes, n_num):
#         super().__init__()
#         self.embeddings = nn.ModuleList([nn.Embedding(categories, size) for categories, size in embedding_sizes])
#         n_emb = sum(e.embedding_dim for e in self.embeddings)
#         self.n_emb, self.n_num = n_emb, n_num
#         self.lin1 = nn.Linear(self.n_emb + self.n_num, 100)
#         self.lin2 = nn.Linear(100, 70)
#         self.lin3 = nn.Linear(70, 2)
#         self.bn1 = nn.BatchNorm1d(self.n_num)
#         self.bn2 = nn.BatchNorm1d(100)
#         self.bn3 = nn.BatchNorm1d(70)
#         self.emb_drop = nn.Dropout(0.6)
#         self.drops = nn.Dropout(0.3)
 
#     # 順伝播
#     def forward(self,x_cat,x_num):
#         x = [e(x_cat[:, i]) for i, e in enumerate(self.embeddings)]
#         x = torch.cat(x, dim=1)
#         x = self.emb_drop(x)
#         x2 = self.bn1(x_num)
#         x = torch.cat([x, x2], dim=1)
#         x = F.relu(self.lin1(x))
#         x = self.drops(x)
#         x = self.bn2(x)
#         x = F.relu(self.lin2(x))
#         x = self.drops(x)
#         x = self.bn3(x)
#         x = self.lin3(x)
#         return x

In [41]:
# # ラベルエンコード済みカテゴリ変数の埋め込み
# # 各カテゴリ列の変数の種類
# cat_sizes = [len(df_train[col].cat.categories) for col in CATEGORICAL]

# # (入力サイズ, 50と割る2の小さい方)でエンコード
# emb_sizes = [(size, min(50, (size+1)//2)) for size in cat_sizes]

In [42]:
# # 記録用
# hist = {
#     'train_loss': [], 'train_auc': [],
#     'val_loss': [], 'val_auc': []
# }

# # パラメータ
# bs = 64 # バッチサイズ
# EPOCHS = 5 # エポック
# save_every = 1
# FOLDS = 5 # FOLD数
# LR=1e-3 # 学習率

# patience = 3

# # stratified KFoldの宣言
# skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)

# fold_results = []

# val_results = []
# test_results = []

# # SKFによるデータ分割
# for fold, (train_idx, val_idx) in enumerate(skf.split(df_train.drop(columns=TARGET), df_train[TARGET])):
    
#     print(f"\n========== Fold {fold+1} ==========")

#     # 学習データ
#     X_train = df_train.drop(columns=TARGET).iloc[train_idx] 
#     y_train = df_train[TARGET].iloc[train_idx]

#     # バリデーションデータ
#     X_val = df_train.drop(columns=TARGET).iloc[val_idx]
#     y_val = df_train[TARGET].iloc[val_idx]

#     # Datasetの作成
#     train_dataset = CustomDataset(X_train, y_train)
#     val_dataset = CustomDataset(X_val, y_val)
    
#     # DataLoaderの作成
#     train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True, num_workers=0)
#     val_loader = DataLoader(val_dataset, batch_size=bs, shuffle=False, num_workers=0)

#     # モデル構築
#     model = NN_Model(emb_sizes, len(NUMERICAL)).to(device)

#     # 最適化設定
#     optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-5)
#     # optimizer = torch.optim.Adam(model.parameters(), lr=LR)

#     # 損失関数
#     criterion = nn.CrossEntropyLoss()

#     hist = {"train_auc": [], "val_auc": []}
#     best_val_auc = 0
#     counter = 0

#     # 学習・予測エポックのループ
#     for epoch in range(EPOCHS):

#         # 開始時間
#         start_time = time.time()

#         # 学習モード
#         model.train()

#         # ラベル、予測値の保存場所
#         y_true_train, y_pred_train = [], []

#         # プログレスバー
#         train_iter = tqdm(train_loader, desc=f"<Train> Epoch {epoch+1}", leave=False)
        
#         for i, (cat_data, num_data, target) in enumerate(train_iter):

#             # DataLoaderから取り出した、カテゴリ、数値、ターゲット
#             cat_data, num_data, target = cat_data.to(device), num_data.to(device), target.to(device)

#             # パラメータの勾配を初期化
#             optimizer.zero_grad()

#             # 予測値の算出
#             output = model(cat_data, num_data)

#             # ラベルと予測値とのロス計算
#             loss = criterion(output, target)

#             # 各パラメータの勾配を算出
#             loss.backward()

#             # パラメータ更新
#             optimizer.step()

#             # ソフトマックスの分類結果を格納
#             probs = torch.softmax(output, dim=1)[:, 1].detach().cpu().numpy()
#             y_pred_train.extend(probs)

#             # ラベルの格納
#             y_true_train.extend(target.cpu().numpy())

#             # プログレスバーの後ろにロス値を表示
#             if i % 10 == 0:
#                 train_iter.set_postfix(loss=loss.item())

#         # チェックポイント保存
#         if (epoch + 1) % save_every == 0:
#             torch.save(model.state_dict(), f"model_epoch{epoch+1}.pt")        
    
#         # histに残すAUCスコア
#         train_auc = roc_auc_score(y_true_train, y_pred_train)

#         # 評価モード
#         model.eval()

#         # ラベル、予測値の保存場所        
#         y_true_val, y_pred_val = [], []

#         # プログレスバー
#         val_iter = tqdm(val_loader, desc=f"<Val> Epoch {epoch+1}", leave=False)

#         # 勾配を更新しない
#         with torch.no_grad():
            
#             for cat_data, num_data, target in val_iter:
    
#                 # DataLoaderから取り出した、カテゴリ、数値、ターゲット
#                 cat_data, num_data, target = cat_data.to(device), num_data.to(device), target.to(device)

#                 # 予測値の算出
#                 output = model(cat_data, num_data)

#                 # ソフトマックスの分類結果を格納
#                 probs = torch.softmax(output, dim=1)[:, 1].cpu().numpy()
#                 y_pred_val.extend(probs)

#                 # ラベルの格納
#                 y_true_val.extend(target.cpu().numpy())

#                 # プログレスバーの後ろにロス値を表示
#                 val_iter.set_postfix(loss=criterion(output, target).item())

#         # histに残すAUCスコア        
#         val_auc = roc_auc_score(y_true_val, y_pred_val)

#         # 差分時刻
#         elapsed = time.time() - start_time

#         # 履歴追加
#         hist["train_auc"].append(train_auc)
#         hist["val_auc"].append(val_auc)

#         # 進捗
#         print(f"Epoch {epoch+1}/{EPOCHS} - TrainAUC: {train_auc:.4f} | ValAUC: {val_auc:.4f} | Time: {elapsed:.1f}s")

#         # チェックポイント
#         if (epoch + 1) % save_every == 0:
#             torch.save(model.state_dict(), f"model_fold{fold+1}_epoch{epoch+1}.pth")

#         # EarlyStopping判定
#         if val_auc > best_val_auc:
#             best_val_auc = val_auc
#             counter = 0
#             torch.save(model.state_dict(), f"best_model_fold{fold+1}.pth")
#         else:
#             counter += 1
#             if counter >= patience:
#                 print(f"Early stopping at epoch {epoch+1}")
#                 break

    
#     # foldごとに保存
#     torch.save(model.state_dict(), f"model_fold{fold+1}.pth")
    
#     # ヒストグラムの更新
#     fold_results.append(hist)

#     # foldごとにテストデータ計算
#     model.eval()
#     with torch.no_grad():
#         X_val_cat = torch.from_numpy(df_train[CATEGORICAL].values.astype(np.int64)).to(device)
#         X_val_num = torch.from_numpy(df_train[NUMERICAL].values.astype(np.float32)).to(device)

#         # 予測
#         preds = torch.softmax(model(X_val_cat, X_val_num),dim=1)[:,1].cpu().numpy()
#         val_results.append(preds)
        
#         X_test_cat = torch.from_numpy(df_test[CATEGORICAL].values.astype(np.int64)).to(device)
#         X_test_num = torch.from_numpy(df_test[NUMERICAL].values.astype(np.float32)).to(device)

#         # 予測
#         preds = torch.softmax(model(X_test_cat, X_test_num),dim=1)[:,1].cpu().numpy()
#         # preds = torch.softmax(model(X_test_cat, X_test_num).squeeze()).cpu().numpy()
#         test_results.append(preds)
    
# # shape = (n_folds, n_test_samples) → 平均化
# val_results = np.mean(val_results, axis=0)        
# test_results = np.mean(test_results, axis=0)        

In [43]:
# pred_NN = val_results
# pred_NN_test = test_results
# AUC_NN = np.average(hist["val_auc"])

In [44]:
# # ======================
# # FoldごとのAUCをプロット
# # ======================
# import matplotlib.pyplot as plt
# plt.figure(figsize=(10,5))
# for i, hist in enumerate(fold_results):
#     plt.plot(hist["val_auc"], label=f"Fold {i+1} Val AUC")
# plt.xlabel("Epoch")
# plt.ylabel("AUC")
# plt.legend()
# plt.title("Validation AUC per Fold")
# plt.show() 

In [45]:
# from sklearn.ensemble import StackingClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import cross_val_score, StratifiedKFold
# from sklearn.metrics import roc_auc_score

# print("# STACKING ENSEMBLE IMPLEMENTATION")
# print("# " + "="*50)
# print("# Combining top 4 models: LightGBM, XGBoost, CatBoost, NN")
# print("# Meta-learner: Logistic Regression")
# print("# " + "="*50)

# stacking_train = pd.DataFrame({
#     'lgb': pred_lgb,
#     'xgb': pred_xgb, 
#     'cat': pred_cb,
#     'NN': pred_NN,
# })

# stacking_test = pd.DataFrame({
#     'lgb': pred_lgb_test,
#     'xgb': pred_xgb_test,
#     'cat': pred_cb_test,
#     'NN': pred_NN_test,
# })

# print(f"# Stacking train shape: {stacking_train.shape}")
# print(f"# Stacking test shape: {stacking_test.shape}")

# print("\n# METHOD 1: WEIGHTED AVERAGE")
# print("# " + "-"*30)

# scores = [AUC_lgb, AUC_xgb, AUC_cb, AUC_NN]  
# total_score = sum(scores)
# weights = [score/total_score for score in scores]

# print(f"# Model weights:")
# print(f"# LightGBM: {weights[0]:.4f}")
# print(f"# XGBoost:  {weights[1]:.4f}")
# print(f"# CatBoost: {weights[2]:.4f}")
# print(f"# NN: {weights[3]:.4f}")

# weighted_oof = (stacking_train['lgb'] * weights[0] + 
#                 stacking_train['xgb'] * weights[1] + 
#                 stacking_train['cat'] * weights[2] +
#                 stacking_train['NN'] * weights[3])

# weighted_test = (stacking_test['lgb'] * weights[0] + 
#                  stacking_test['xgb'] * weights[1] + 
#                  stacking_test['cat'] * weights[2] +
#                  stacking_test['NN'] * weights[3])

# weighted_score = roc_auc_score(y, weighted_oof)
# print(f"# Weighted Average ROC AUC: {weighted_score:.6f}")

# print("\n# METHOD 2: LOGISTIC REGRESSION META-LEARNER")
# print("# " + "-"*40)

# meta_learner = LogisticRegression(penalty="l2",random_state=42, max_iter=1000)
# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# cv_scores = cross_val_score(meta_learner, stacking_train, y, 
#                            cv=skf, scoring='roc_auc', n_jobs=-1)

# print(f"# Meta-learner CV scores: {[f'{score:.6f}' for score in cv_scores]}")
# print(f"# Meta-learner mean CV: {cv_scores.mean():.6f} ± {cv_scores.std():.6f}")

# meta_learner.fit(stacking_train, y)
# meta_oof = meta_learner.predict_proba(stacking_train)[:, 1]
# meta_test = meta_learner.predict_proba(stacking_test)[:, 1]
# meta_score = roc_auc_score(y, meta_oof)

# print(f"# Meta-learner ROC AUC: {meta_score:.6f}")

# coefficients = meta_learner.coef_[0]
# print(f"# Meta-learner coefficients:")
# print(f"# LightGBM: {coefficients[0]:.4f}")
# print(f"# XGBoost:  {coefficients[1]:.4f}")
# print(f"# CatBoost: {coefficients[2]:.4f}")
# print(f"# NN: {coefficients[3]:.4f}")
# print(f"# Intercept: {meta_learner.intercept_[0]:.4f}")

# print("\n# METHOD 3: SIMPLE AVERAGE (BASELINE)")
# print("# " + "-"*35)

# simple_oof = (stacking_train['lgb'] + stacking_train['xgb'] + stacking_train['cat'] + stacking_train['NN']) / 4
# simple_test = (stacking_test['lgb'] + stacking_test['xgb'] + stacking_test['cat'] + stacking_test['NN']) / 4
# simple_score = roc_auc_score(y, simple_oof)

# print(f"# Simple Average ROC AUC: {simple_score:.6f}")

# print("\n# ENSEMBLE METHODS COMPARISON")
# print("# " + "="*40)
# ensemble_results = [
#     ('Individual LightGBM', AUC_lgb),
#     ('Individual XGBoost', AUC_xgb),
#     ('Individual CatBoost', AUC_cb),
#     ('Individual NN', AUC_NN),
#     ('Weighted Average', weighted_score),
#     ('Meta-learner (LogReg)', meta_score),
#     ('Simple Average', simple_score)
# ]

# ensemble_results.sort(key=lambda x: x[1], reverse=True)

# for i, (method, score) in enumerate(ensemble_results, 1):
#     print(f"# {i}. {method:<25}: {score:.6f}")

# best_method, best_score = ensemble_results[0]
# print(f"\n# BEST ENSEMBLE METHOD: {best_method}")
# print(f"# BEST ENSEMBLE SCORE: {best_score:.6f}")

# if 'Meta-learner' in best_method:
#     final_oof = meta_oof
#     final_test = meta_test
#     print("# Using Meta-learner predictions for final submission")
# elif 'Weighted' in best_method:
#     final_oof = weighted_oof
#     final_test = weighted_test
#     print("# Using Weighted Average predictions for final submission")
# else:
#     final_oof = simple_oof
#     final_test = simple_test
#     print("# Using Simple Average predictions for final submission")

# print("\n# STACKING ENSEMBLE COMPLETED!")
# print("# " + "="*50)

In [46]:
# # 提出データ作成
# sample_submission = pd.read_csv("/kaggle/input/playground-series-s5e8/sample_submission.csv")

# sample_submission['y'] = test_results
# sample_submission.to_csv('submission.csv', index=False)
# print('Submission file saved.')